<a href="https://colab.research.google.com/github/jessica-hoffman/transformer_practice/blob/main/day8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Day 8 – Debugging & Problem Solving (2 hrs)

**Goal:** Be able to fix errors quickly.

---

## 📖 Reading (~30 min)

### 1. Common PyTorch Errors
- **Device mismatch (`RuntimeError: Expected all tensors to be on the same device`)**
  - Happens if model is on GPU but data is still on CPU (or vice versa).
  - Fix: `.to(device)` on both model and tensors.

- **GradScaler warnings (AMP training)**
  - AMP = Automatic Mixed Precision.
  - Errors like `GradScaler has been disabled` → usually because loss became NaN.
  - Fix: reduce learning rate, check for exploding gradients, clip gradients.

- **Shape mismatch**
  - Example: logits shape `[batch_size, vocab_size]` vs labels shape `[batch_size, seq_len]`.
  - Fix: make sure labels align with model output (e.g. shift labels in language modeling).

---

### 2. Hugging Face Specific Errors
- **`KeyError: 'input_ids'`**
  - Hugging Face models expect inputs in dicts with keys: `input_ids`, `attention_mask`, etc.
  - Fix: make sure your tokenizer output is passed as `**batch`.

- **`ValueError: Tokenizer does not have a pad_token`**
  - Needed for batching sequences of different lengths.
  - Fix:
    ```python
    tokenizer.pad_token = tokenizer.eos_token
    ```

- **`RuntimeError: CUDA out of memory`**
  - Fixes: reduce batch size, use gradient accumulation, switch to smaller model, use `.half()`.

---

## 🛠 Hands-on (~1.5 hr)

We’ll **intentionally introduce errors** and practice debugging.

### 1. Device mismatch




In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = nn.Linear(10, 2).to(device)
x = torch.randn(4, 10)   # Oops! Still on CPU

# ❌ Will error:
out = model(x)

RuntimeError: Expected all tensors to be on the same device, but got mat1 is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA_addmm)

Fix:

In [2]:
x = x.to(device)
out = model(x)

2. KeyError: 'input_ids'

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

text = ['hello world']
batch = tokenizer(text, padding=True, return_tensors='pt')

# ❌ Mistakenly try to access wrong key
print(batch["ids"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

KeyError: 'ids'

Fix:

In [4]:
print(batch["input_ids"])

tensor([[ 101, 7592, 2088,  102]])


3. Tokenizer pad_token issue

In [5]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('gpt2')
tok.pad_token = None

# ❌ This will error because GPT-2 has no pad_token
batch = tok(["hi", "there"], padding=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

Fix:

In [6]:
tok.pad_token = tok.eos_token
batch = tok(['hi', 'there'], padding=True, return_tensors='pt')

4. Shape mismatch

In [7]:
logits = torch.randn(4,5)   # (batch, classes)
labels = torch.randint(0, 5, (4,3)) # (batch, seq_len) ❌ mismatch

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

Fix:

In [8]:
labels = torch.randint(0, 5, (4,)) # match (batch,)
loss = loss_fn(logits, labels)

5. AMP / GradScaler

In [9]:
scaler = torch.cuda.amp.GradScaler()
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

for i in range(100):
    with torch.cuda.amp.autocast():
        out = model(torch.randn(32, 10).to(device))
        loss = out.mean() * 1e6  #  # ❌ artificial overflow
    scaler.scale(loss).backward()
    scaler.step(optim)
    scaler.update()

/tmp/ipython-input-1992223534.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipython-input-1992223534.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Fixes:

Lower learning rate.

Clip gradients.

Avoid extreme scaling.

In [10]:
scaler = torch.cuda.amp.GradScaler()
optim = torch.optim.Adam(model.parameters(), lr=1e-5)   # lower learning rate

for i in range(100):
    with torch.cuda.amp.autocast():
        out = model(torch.randn(32, 10).to(device))
        loss = out.mean()                       # remove the high scaling coefficient that causes overflow
    scaler.scale(loss).backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)        # gradient clipping right before step - this keeps gradient from exploding
    scaler.step(optim)
    scaler.update()


/tmp/ipython-input-2604067658.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipython-input-2604067658.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
